In [11]:
import os
from src.solver import DetSolver
from src.core import YAMLConfig
import torch

config_path = 'configs/rtdetr/rtdetr_r18vd_6x_coco.yml'
checkpoint_url = "https://github.com/lyuwenyu/storage/releases/download/v0.1/rtdetr_r18vd_dec3_6x_coco_from_paddle.pth"

torch.hub.load_state_dict_from_url(checkpoint_url, "../models");

name = os.path.basename(checkpoint_url)
checkpoint_path = f"../models/{name}"

In [1]:
import os
from src.solver import DetSolver
from src.core import YAMLConfig
import torch

config_path = '../models/taco/config.yml'
checkpoint_path = '../models/taco/checkpoint0199.pth'

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
cfg = YAMLConfig(
    config_path,
    resume=checkpoint_path,
    # use_amp=args.amp,
    tuning=''
)

solver = DetSolver(cfg)

In [13]:
solver.setup()
solver.resume(solver.cfg.resume)

Load PResNet18 state_dict
Loading ema.state_dict


In [4]:
from src.data.transforms import Resize, ToImageTensor, ConvertDtype, Compose
from PIL import Image, ImageOps

t = Compose(ops=[
    Resize([640, 640]),
    ToImageTensor(),
    ConvertDtype()
])

In [5]:
path = "image_02.jpg"
path = "taco.jpg"

img = Image.open(path).convert("RGB")

In [6]:
try:
    img = ImageOps.exif_transpose(img)
except:
    print("No EXIF data")
    pass

w, h = img.size
orig_target_sizes = torch.as_tensor([int(w), int(h)]).unsqueeze(0)

In [7]:
device = "cuda:0"
model = solver.model
# model = solver.ema.module if solver.ema else solver.model
model.eval()
postprocessors = solver.postprocessor
postprocessors.num_top_queries = 5
# dataloader = None
with torch.no_grad():
    samples = t(img)[None]
    samples = samples.to(device)
    orig_target_sizes = orig_target_sizes.to(device)
    outputs = model(samples)
    results = postprocessors(outputs, orig_target_sizes)

In [15]:
import supervisely as sly
img = sly.image.read(path)
for box in results[0]['boxes'].cpu().numpy():
    x, y, x2, y2 = box
    sly.Rectangle(top=y, left=x, bottom=y2, right=x2).draw_contour(img, color=[255, 0, 0], thickness=3)
sly.image.write("out.jpg", img)
results

[{'labels': tensor([9, 8, 9, 0, 8], device='cuda:0'),
  'boxes': tensor([[ 9.4745e+02,  1.8027e+03,  1.4819e+03,  2.1311e+03],
          [ 6.6153e+02, -1.0591e+01,  3.2830e+03,  1.9432e+03],
          [-1.1040e+00,  1.6936e+00,  8.3034e+02,  1.4433e+03],
          [ 2.3414e+03,  1.3992e+03,  2.3633e+03,  1.4223e+03],
          [ 4.4925e+01, -7.1199e+00,  3.3019e+03,  1.9414e+03]], device='cuda:0'),
  'scores': tensor([0.9808, 0.0624, 0.0450, 0.0402, 0.0385], device='cuda:0')}]

In [39]:
from src.data.coco.coco_dataset import mscoco_category2name
mscoco_category2name